In [2]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [2]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [1]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-03-12 14:58:42.135204: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 14:58:42.201763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-12 14:58:42.201784: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:

nActions = 4
tableSize = 4
hiddenFeatures = 16
temporalWindow = 5
env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))
model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha
# model.add(Activation(activation="elu")) # Tune alpha

model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()
replayMemory = SequentialMemory(limit=100, window_length=temporalWindow)
dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=100,
    target_model_update=0.01,
    policy=policy
)
dqn.compile(optimizer=Adam(lr=0.001), metrics=["mae"])
dqn.fit(env=env, nb_steps=5000, visualize=False, verbose=2)


/usr/local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2025-03-12 15:04:45.737905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-12 15:04:45.737971: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-12 15:04:45.737990: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (923b5dc5ef02): /proc/driver/nvidia/version does not exist
2025-03-12 15:04:45.738867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX5

Training for 5000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  146/5000: episode: 1, duration: 0.764s, episode steps: 146, steps per second: 191, episode reward: 288.000, mean reward:  1.973 [ 0.000, 32.000], mean action: 2.726 [0.000, 3.000],  loss: 17.230431, mae: 5.464291, mean_q: 12.271279
  445/5000: episode: 2, duration: 1.824s, episode steps: 299, steps per second: 164, episode reward: 1244.000, mean reward:  4.161 [ 0.000, 132.000], mean action: 1.656 [0.000, 3.000],  loss: 57.038853, mae: 20.151827, mean_q: 32.551838
  823/5000: episode: 3, duration: 2.299s, episode steps: 378, steps per second: 164, episode reward: 928.000, mean reward:  2.455 [ 0.000, 72.000], mean action: 1.548 [0.000, 3.000],  loss: 42.011642, mae: 47.876934, mean_q: 73.160934
 1077/5000: episode: 4, duration: 1.523s, episode steps: 254, steps per second: 167, episode reward: 964.000, mean reward:  3.795 [ 0.000, 76.000], mean action: 1.035 [0.000, 3.000],  loss: 51.596786, mae: 52.684246, mean_q: 78.994682
 1309/5000: episode: 5, duration: 1.377s, episode steps: 23

# Simulate Game

In [8]:
# Play the game with predicts

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)
        predictedAction = dqn.forward(observation)
        print("predictedAction")
        print(predictedAction)
        # currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]

        # game.move(currentMove)
        # print(f"Game {i + 1} after move '{currentMove}':")
        # game.render()
        break

    print(f"Game {i + 1} is over.")


Game 1 initial state:
Score: 0
[[0 0 0 2]
 [0 0 2 0]
 [0 0 0 0]
 [0 0 0 0]]


ValueError: Error when checking input: expected flatten_input to have 4 dimensions, but got array with shape (1, 5)

# Manual Game

In [ ]:
moves = ['up', 'left', 'right', 'down']
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


